<script>
  jQuery(document).ready(function($) {

  $(window).load(function(){
    $('#preloader').fadeOut('slow',function(){$(this).remove();});
  });

  });
</script>

<style type="text/css">
  div#preloader { position: fixed;
      left: 0;
      top: 0;
      z-index: 999;
      width: 100%;
      height: 100%;
      overflow: visible;
      background: #fff url('http://preloaders.net/preloaders/720/Moving%20line.gif') no-repeat center center;
  }

</style>

<div id="preloader"></div>

<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>

### Latex Macros
$\newcommand{\Re}[1]{{\mathbb{R}^{{#1}}}}
\newcommand{\Rez}{{\mathbb{R}}}$

In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import numpy as np
import networkx as nx
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from scipy.stats import ortho_group  # compute unitary matrices
import matplotlib.pyplot as plt
import copy
import sympy
import spectral_function_library as spec_lib
#%matplotlib inline

# Convolution Neural Networks 
Material is taken from [this Blog](https://www.instapaper.com/read/1477946505)

Starting from an RGB image: 

<img src="attachment:184a4f79-819b-42b2-96e6-6123f3a0cb26.png" width="800">

the idea is pass this image through as series of steps in order to extract information. The filter is used for this task. 

<img src="attachment:e6bb7f0b-06cb-41c9-b314-09df22cadab1.png" width="800">

after image src.

<img src="attachment:e6bb7f0b-06cb-41c9-b314-09df22cadab1.png" width="800">

<img src="attachment:32eba126-86c4-46d9-8f70-1732e279902a.png" width="800">

<img src="attachment:9c228339-f401-4074-8730-49b67155ac7b.png" width="800">

## Two important points about the convolutional layer: 

1. The filter is identical for each pixel. This reduces the number of parameters to calculate. 
The constant filter helps satisfy the inductive bias of *translation invariance*. 

2. The convolution is local to the image pixel to which it is applied. Thus, the structure of the image is taken into account during the calculation. 

A typical CNN architecture: 

<img src="attachment:c422d8dd-d883-46cf-99ba-5bdf15ac347a.png" width="800">

jupyter nbextension enable --py widgetsnbextensionjupyter nbextension enable --py widgetsnbextensionjupyter nbextension enable --py widgetsnbextensionjupyter nbextension enable --py widgetsnbextension# Alternative view of CNN

<img src="attachment:34feb6d0-71dd-4c44-972b-ae662ee8ce3c.png" width="800">

* An image can be considered to be a graph
* The nodes $V$ are the centers of the pixels
* If a filter has width 3, each nodes is connected to $8 * d$ adjacent nodes, where $d$ is the number of channels

# Motivation
Consider a set of nodes $x_i$, and associated attributes $y_i$. This can be graphed. Let us connect these nodes with edges $e_{ij} = (x_i, x_{i+1})$.

In [3]:
@interact(N=(5,40))
def plot1d(N):
    x = np.linspace(0, 10, N)
    plt.plot(x, 0*x, '-o')
    plt.show()

interactive(children=(IntSlider(value=22, description='N', max=40, min=5), Output()), _dom_classes=('widget-in…

Add an attribute to each of these nodes. I will add a random noise in $N(0,\sigma)$ and $\sigma=1.5$, which is fairly large. 

Consider the problem of computing *embeddings* of each node with the requirement that nearby nodes with similar attributes should have similar embeddings. 

Without further constraints imposed on the problem (also called *inductive biases*, we will apply a local transformation to this function, and specifically an averaging operation. We will replace $y_i$ by the average of its neighbors :
$$ y_i \longrightarrow \frac12 (y_{i-1} + y_{i+1})$$
The boundary points need special treatment. There are three main ehoices: 
1. Do not move the point
2. Move the point in such a way as to satisfy some condition on the slope.
3. Develop an algorithm that figures out the proper treatment

We will consider the first choice for simplicity. For future reference, we call the collection of points $V$, the collection of edges $E$. We denote the boundary nodes by $\partial V$, and the boundary edges (edges attached to $\partial V$ by $\partial E$, which is a common notation in discrete and differential geometry. 

In [3]:
@interact(seed=(1,100), eps=(0,1.5), N=(5,40))
def plot1d(seed, eps, N):
    np.random.seed(seed)
    x = np.linspace(0, 10, N)
    noise = eps * np.random.randn(N)
    y = np.sin((x/x[-1])*2*np.pi*2.5) + noise
    plt.plot(x, y, '-o')
    plt.show()

interactive(children=(IntSlider(value=50, description='seed', min=1), FloatSlider(value=0.75, description='eps…

More generally, each point might have multiple attribute. Thus, the node $x_i$, would have $d$ attributes $y_0, \cdots, y_{d-1}$. These attributes could be categorical or continuous, and the categorical attributes could be nominal (there is nor ordering, such as 'red', 'blue', 'orange') or ordinal (bad, poor, average, good, very good excellent). 

In [4]:
dSlider = widgets.IntSlider(min=1, max=5, value=3, description="Nb Attributes")
seedSlider = widgets.IntSlider(min=1, max=100, value=50, description="Seed")
epsSlider = widgets.FloatSlider(min=0., max=1.5, value=0.30, description="Noise $\sigma$")
@interact(seed=seedSlider, eps=epsSlider, N=(5,40), d=dSlider, nb_blur_iter=(0,5))
def plot1d(seed, eps, N, d, nb_blur_iter):
    np.random.seed(seed)
    eps = eps * np.array([1.,2.,.5,3.,4.])
    x = np.linspace(0, 10, N)
    noise = np.random.randn(d,N)
    y = np.zeros([5, N])
    fcts = {}
    fcts[0] = np.sin((x/x[-1])*2*np.pi*2.5)
    fcts[1] = 1.5 *  np.cos((x/x[-1])*2*np.pi*2.5) ** 2
    fcts[2] = x ** 2 / 10 * np.exp(3-.5*x)
    fcts[3] = np.cos((x/x[-1])*2*np.pi*4.5)
    fcts[4] = 1.5 *  np.cos((x/x[-1])*2*np.pi*2.5)
    for i in range(0,5):
        y[i] = fcts[i]
    
    for i in range(0, d):
        y[i] += eps[i] * noise[i]
        
    yy = copy.copy(y)
    
    for i in range(0, d):
        for n in range(0, nb_blur_iter):
            yy[i][0] = y[i][0]
            yy[i][N-1] = y[i][N-1]
            yy[i][1:N-2] = 0.5 * (y[i][0:N-3] + y[i][2:N-1])
            y = copy.copy(yy)
    
    for i in range(0,d):
        plt.plot(x, yy[i], '-o')
    plt.grid(True)
    plt.ylim(-2, 5)
    plt.show()

interactive(children=(IntSlider(value=50, description='Seed', min=1), FloatSlider(value=0.3, description='Nois…

So far, I am describing vector-valued discrete functions of $x$, which is a 1-D representation of a graph $d$ attributes at each node $x_i$. More generally, nodes are points in *some* space, which can be 1-D, 2-D, higher-D, or more abstract, namely, a space of *points*. 

Now consider adding attributes $y_{Eij}$ to the edges. What kind of transformation functions should one consider? 

This averaging function is an example of a local filter defined in physical space. This filter takes attributes at nodes and transforms them into a new set of number defined at these same nodes. More generally, in Graph Neural networks, we will consider operators that take attributes defined at nodes, edges, and the graph, and transform them into a new set of vectors defined on these same nodes, vectors and graphs. 

Filters can be defined either in physical space or in spectral space. We will illustrate the concept by considering the derivative operator on continuous and discrete grids.

## First Derivative operator (also a filter) on 1D grid in physical space
Consider points $x_i$, $i=0,\cdots, N-1$ connected by edges $e_{i,i+1} = (x_i, x_{i+1})$. The central difference operator of the function $f_i = f(x_i)$ is defined by
$$
f'_i = \frac{f_{i+1} - f_{i-1}}{x_{i+1} - x_{i-1}}
$$ for $i=1,\cdots,N-2$, with one-sided operators defined at the boundaries (which is one of many possibilities): 
\begin{align}
f'_0 &= \frac{f_1 - f_0}{x_1-x_0} \\
f'_{N-1} &= \frac{f_{N-1} - f_{N-2}}{x_{N-1} - x_{N-2}}
\end{align}
where $f'_i$ is the approximation of $f'(x)$ evaluated at $x=x_i$. Note that the derivative can be expressed as a vector 
$f' = (f'_0,\cdots,f'_{N-1})$, and $f'_i$ is linear with respect to the values $f_j$. Therefore one can write the matrix 
expression: 
$$ f' = D f $$
where $D \in \Re{N\times N}$ is an $N \times N$ matrix.  The matrix $D$ is a derivative filter. More specifically, it is a 
*global* filter since it updates the values at all nodes at once. To the contrary, a *local* filter is defined as the matrix that updates the derivative at a single point. Thus: 
$$
f'_i = (\begin{matrix}-\alpha & 0 & \alpha\end{matrix})^T 
   (\begin{matrix} f_{i+1} & 0 & f_{i-1}) \end{matrix}
$$
where a superscript $T$ denotes transpose, and $\alpha = (x_{i+1} - x_{i-1})^{-1}$. Clearly, the local 
filter is local to the point at which it applies. The new value only depends on the values of its immediate neighbors. 

***
# Spectral Analysis of graphs
## Continuous Fourier Transform (CFT)
When working in the continuous domain $\Rez$, a function $f(x)\in\Rez$ has a Fourier Transform $\hat{f}(k)$ related by 
$$  \hat{f}(k) = \frac{1}{2\pi} \int_{-\infty}^\infty e^{\iota k x} f(x) \, dx $$
Conversely, one can apply a similar operation to recover $f(x)$ from its Fourier Transform: 

$$ f(x) = \frac{1}{2\pi} \int_{-\infty}^\infty e^{-\iota k x} \hat{f}(k) \, dk $$

Notice the sign in the exponent: positive when transforming from physical to Fourier space, and negative when returning to physical space. The sign is a convention. Different authors might use the opposite sign. So always pay attention to the conventions in any paper you read. 

(you should all have learned about the Fourier transform previously). 

Let us compute the first derivative of $f(x)$: 
$$\frac{d}{dx} f(x) = f'(x)$$
The conventional approach would be to calculate the derivative manually, or discretize the expression in physical space. However, the alternative is to compute the derivative by first transforming the expression to Fourier (also called spectral) space: 

\begin{align}
 \frac{d}{dx} f(x) &= \frac{d}{dx} \frac{1}{2\pi} \int_{-\infty}^\infty e^{-\iota k x} \hat{f}(k)  d k  \\ 
 &=  \frac{1}{2\pi} \int_{-\infty}^\infty (-\iota k) e^{-\iota k x} \hat{f}(k) dk \\
 &= \cal{F}^{-1} [-\iota  k \hat{f}(k)]
\end{align}
where 
\begin{align}
\cal{F}f(x) &= \hat{f}(k) \\
\cal{F}^{-1} \hat{f}(k) &= f(x) \\
\end{align}

So to given a function $f(x)$, one can compute the derivative with the following three steps: 
1.  $f(x) \longrightarrow \hat{f}(k)$
2.  $\hat{f}(k) \longrightarrow (-\iota k) \hat{f}(k)$
3.  $(-\iota k)\hat{f}(k) \longrightarrow \cal{F}^{-1} \left[(-\iota k)\hat{f}(k)\right] = \frac{d}{dx} f(x)$

Thus, the derivative operation is applied in Fourier space. A complex operation in physical space becomes a simple multiplication in Fourier space, *at the cost* of two Fourier Transforms. 

### Fourier Spectrum
$\hat{f}(k)$ is called the Fourier Spectrum and is generally a complex variable. 
$P(k) = |\hat{f}(k)|^2$ is the power spectrum, and satisfies the property: 
$$
\int_{-\infty}^\infty P(k) dk = \int_{-\infty}^\infty |\hat{f}(k)|^2 dx = \int_{-\infty}^\infty |f(x)|^2 dx
$$
a rule that generalizes to and holds in $\Re{n}$. 

### Filter
The coefficient $(-\iota k)$ above is an example of a complex operator in Fourier space. This operator tranforms a function $\hat{f}(k)$ into a "filtered" function $\hat{g}(k)$: 
$$
\hat{g}(k) = (-\iota k) \hat{f}(k)
$$
and in this particular case, results in the Fourier transform of the $x$-derivative of $f(x)$. More generally, one can define an operator $\hat{H}(k)$ acting on $\hat{f}(k)$, which "shapes" the power spectrum, leading to filters with different characteristics: low-pass, band-pass, high-pass, custom. 

Given a function $f(x)$, the resulting filtered function $f_H(x)$ can be defined similarly to the derivative: 

\begin{align}
 f(x) & \longrightarrow \cal{F}(f(x)) = \hat{f}(k) \\
  \hat{f}(k) & \longrightarrow \hat{H}(k) \hat{f}(k) \\
 \hat{H}(k)\hat{f}(k) & \longrightarrow \cal{F}^{-1} (\hat{H}(k)\hat{f}(k)) = f_H(x)
\end{align}

We will often omit the argument $x$ or $k$, letting the "hat" notation indicate whether or not we are in Fourier space. Thus, we can write
$$
f_H = \cal{F}^{-1} [\hat{H} \; \cal{F}(f) ]
$$ or the equivalent form (requiring the definition of product of operators): 

\begin{align}
f_H &= (\cal{F}^{-1} \, \hat{H} \, \cal{F}) \; f \\
   &= H f
\end{align}
which defines the filter $H(x)$ in physical space, acting on $f(x)$ to produce $f_H(x)$: 
$$
f_H(x)   = H(x) * f(x)
$$
where $*$ denotes the convolution operator: 
$$
H(x) * f(x) = \int_{-\infty}^\infty H(x-s) f(s) \, ds$$

## Interactive examples of filters in continuous space
We build upon the description at http://localhost:8889/lab/tree/Spectral.ipynb

Consider the function $f(x) = \sum_i 

COMPLETE LATER.

# Ideal Low-, Mid-, High-pass filters
## Low-pass filter

\begin{align}
H(k) &= 1, \hspace{1in} k < k_0 \\
&= 0, \hspace{1in} k \ge k_0 
\end{align}
## Band-pass filter

\begin{align}
H(k) &= 1, \hspace{1in} k_0 < k < k_1, \; k_0 < k_1 \\
&= 0  \hspace{1in} \rm{otherwise}
\end{align}
## High-pass filter

\begin{align}
H(k) &= 1, \hspace{1in} k > k_0 \\
&= 0, \hspace{1in} k \le k_0 
\end{align}


#### Notes: 
* np.fft uses the discrete Fourier Tranform since the grid is discrete (we skip over these details)
* The $x$-domain is $[0,0.5]$. 
* $\sin(2\pi f_1 x)= 0$ at $x=0$ and $x=0.5$. The $x-derivative is $2\pi f_1\cos(f_1 2\pi x)$, equal 
to $2\pi f_1$ at $x=0$ and $2\pi f_1 \cos(\pi f_1)$ at $x=0.5$, equal to 2\pi f_1$ if $f_1$ is even. 
Therefore the function is  periodic over the domain, since the $f_1$ slider ranges from -40 to 40 by increments of 10.  
On the other hand, $\cos(2\pi f_3 x + 0.7)$ is not periodic over the $x$ domain (the phase is 0.7, which is not a multiple of $2\pi$. The frequencies are obtained by 
decomposing this function into a series of $\sin$ and $\cos$ at different frequencies with zero phase. 

In [8]:
@interact_manual(
    freq1=(-20, 60, 10),
    freq2=(-90, 90, 10),
    freq3=(-300, 300, 15),
    ampl1=1,
    ampl2=0.5,
    ampl3=1,
    k0=(0, 50, 5),
    k1=(5, 150, 10),
)
def plotSin2(freq1, freq2, freq3, ampl1, ampl2, ampl3, k0, k1):
    fig = plt.figure(figsize=(16, 7))
    x = np.linspace(0, 0.5, 500)
    k = np.linspace(0, 499, 500)
    
    # NOTE: These functions are NOT periodic over the domain. 
    # Therefore, the spectrum is not exactly a collection of delta functions
    # I could be more precise, but that is not the point of this demonstration. 
    
    s = (
        ampl1 * np.sin(freq1 * 2 * np.pi * x)
        + ampl2 * np.sin(freq2 * 2 * np.pi * x)
        + ampl3 * np.cos(freq3 * 2 * np.pi * x + 0.7)
    )
    nrows, ncols = 3, 2

    # ax1.clear()  # to avoid flicker, does not work
    ax = fig.add_subplot(nrows, ncols, 1)
    # fig, axes = plt.subplots(nrows, ncols, figsize=(16, 5))
    ax.set_ylabel("Amplitude")
    ax.set_xlabel("Time [s]")
    ax.plot(x, s)

    fft = np.fft.fft(s)
    ifft = np.fft.ifft(s)
    # print("s: ", s[0:10])
    # print("ifft: ", ifft[0:11])
    # print("fft[0-10]: ", fft[0:11])
    # print("fft[:-10,:]: ", fft[-10:])
    power_spec = np.abs(fft) ** 2
    #power_spec[0] = 0 # REMOVE MEAN COMPONENT  (simply equal to the mean of the function)
    ax2 = fig.add_subplot(nrows, ncols, 2)
    ax = ax2
    ax.plot(power_spec[0 : 250])
    ax.set_ylabel("Power Spectrum")
    ax.set_xlabel("k")

    heaviside = np.where((k > k0) & (k < k1), 1, 0)
    # Symmetrize this function with respect to $k=500/2$
    for i in range(1,250): # 250 = 500/2
        heaviside[500-i] = heaviside[i]  # in Fourier space
    # print(heaviside)

    filtered_power_spectrum = power_spec * heaviside
    # print(list(zip(power_spec, heaviside, filtered_power_spectrum)))
    # print("power spec: ", power_spec[0:50])
    # print("filtered_spec: ", filtered_power_spectrum[0:50])
    filtered_function = np.fft.ifft(filtered_power_spectrum)
    
    ax = fig.add_subplot(nrows, ncols, 3)
    ax.plot(filtered_function)
    ax.set_ylabel("Filtered $f_H(x) = H(x) f(x)$")
    ax.set_xlabel("x")
    
    ax = fig.add_subplot(nrows, ncols, 4)
    ax.plot(filtered_power_spectrum[0 : 250])
    ax.set_xlabel("k")
    ax.set_ylabel("Filtered Power Spectrum")
    
    filter_phys = np.fft.ifft(heaviside)
    ax = fig.add_subplot(nrows, ncols, 5)
    ax.plot(filter_phys)
    ax.set_ylabel("Filter $H(x)$")
    ax.set_xlabel("k")
    
    ax = fig.add_subplot(nrows, ncols, 6)
    ax.plot(heaviside[0:250])
    ax.set_ylabel("Filter $\hat{H}(k)$")
    ax.set_xlabel("k")

    plt.tight_layout()
    plt.show()
    sumf2 = np.sum(s ** 2)
    sump2 = np.sum(power_spec[0:250])
    sump3 = np.sum(power_spec)
    # print(sum2, sump2, sump2 / sumf2, sump3 / sumf2)
    # print(np.sum(power_spec[0:250]), np.sum(power_spec[0:500]), power_spec.shape)

    # The ratio sump2 / sumf2 = 250 (when there is no mean component)
    # The k=0 component has no complex conjugate. All other components have a complex conjugate.
    # These details are beyond the scope of this lecture.
    #     = Number of points N / 2
    # sum f[i]^2 dx = sum f[i]^2 (0.5/N) = sum power_spectrum  * normalizing constant
    #  (one must be careful with this constant)


# Alternative to @interact
# interact(plotSin2, freq1=(-40,40,10), freq2=(-90,90,10), freq3=(-300,300,15), ampl1=1, ampl2=.5, ampl3=1)

interactive(children=(IntSlider(value=20, description='freq1', max=60, min=-20, step=10), IntSlider(value=0, d…

The strong oscilations in the Filter $H(x)$ are due to the discontinuity of the filter in Fourier space. 
A property of these 1-D filters is that localization in Fourier space (the filter is nonzero for very few $k$) leads 
to non-local filters $H(x)$ in physical space, and vice-versa. 

The challenge is to construct filters local in both physical and Fourier space, which is the strength of wavelets (beyond the scope of these lectures). Note that the Fourier transform of a Gaussian is a Gaussian, and it is local in both spaces. (Demonstrate it for yourself as a homework exercise). 

Hi Toan,

I just found out that you never switched your major to an M.S. project. Please do this ASAP. Otherwise you might have issues regarding graduation this semester or this summer. Karey can help you if needed. But you must ask. Thanks. This is your responsibility.

 Gordon## Graphs
The challenge is to generalize the filter Fourier Transforms and Filters to graphs, which are unstructured. There is no particular connectivity patterns as there is for the discrete case of the 1D continuous domain. 

### Discrete 1D domain
* A set of nodes $x_i$, $i=0,1,\cdots,N-1$, such that $x_i$ is connected to $x_{i+1}$. This graph is acyclic (there are no cycles. 
* If the first and last node are connected, we add the edge $(x_{N-1}, x_{0})$ and create a cyclic graph.
* The adjacency matrix of the cyclic graph is as follows: 
$$
A = \left(\begin{matrix}
0 & 0 & 0 & \cdots & 0 & 1 \\
1 & 0 & 0 & \cdots & 0 & 0 \\
0 & 1 & 0 & \cdots & 0 & 0 \\
0 & 0 & 1 & \cdots & 0 & 0 \\
\cdots
\end{matrix}\right)
$$
* A signal $s$ on a graph is defined as the sequence of $N$ elements
$$ x = (x_0, x_1, \cdots, x_{N-1}) $$
where each $x_i\in\Rez$.

Hi Toan,

I just found out that you never switched your major to an M.S. project. Please do this ASAP. Otherwise you might have issues regarding graduation this semester or this summer. Karey can help you if needed. But you must ask. Thanks. This is your responsibility.

 Gordon### 1-D Periodic Domain
#### Fourier Filter
### 1-D Non-periodic Domain
## Fourier Transform, Discrete (DFT)
### 1-D Periodic Domain 
### 1-D Non-periodic Domain
## Graph Signal Processing, Discrete
### 1-D cyclic graph
### 2=D Discrete periodic 
### Adjoint $A$
### Degree Matrix $D$
### Laplacian $L$
###

In [ ]:
# layout = ['circular','planar','random']
seed_slider = widgets.IntSlider(min=100, max=120, step=2, value=110)
N_slider = widgets.IntSlider(min=5, max=40, step=1, value=10)
#matrix = ['Adjacency Matrix', 'Laplacian', 'D^-1 A', 'D^-1 L', 'D^-1/2 L D^-1/2']

@interact(N=N_slider, seed=seed_slider)
def generate_graph_from_adjacency_matrix(N, seed):
    """
    Arguments
    N: number of nodes
    """
    np.random.seed(seed)
    ints = np.random.randint(0, 2, N * N).reshape(N, N)

    # Symmetric array
    ints = ints + ints.transpose()
    ints = np.clip(ints, 0, 1)

    # Different matrices
    A = ints
    D = np.sum(A, axis=0)
    D = np.diag(D)
    L = D - A
    invD = np.linalg.inv(D)
    invDA = A * invD
    invDL = invD * L
    invDLinvD = np.sqrt(invD) * L * np.sqrt(invD)
    
    matrix = ['A', 'D', 'L', 'invD', 'invDA', 'invDL', 'invDinvD']
    matrices = [A, D, L, invD, invDA, invDL, invDLinvD]
    
    # Eigenvalues
    fig, axes = plt.subplots(3,3, figsize=(10,8))
    axes = axes.reshape(-1)
    fig.suptitle("Eigenvalues of various matrices")
    for i,m in enumerate(matrices): 
        ax = axes[i]
        eigs = np.linalg.eigvals(m)
        eigs = np.sort(eigs)[::-1] 
        ax.set_title(matrix[i])
        ax.grid(True)
        ax.plot(eigs, '-o')
    for i in range(i+1,axes.shape[-1]): axes[i].axis('off')
    plt.tight_layout()
    plt.show()


interactive(children=(IntSlider(value=10, description='N', max=40, min=5), IntSlider(value=110, description='s…

***
## Same plot as above but allowing for different types of graph types. 
* Generate the graph, compute the adjacent matrix, and call the previous function



In [9]:
def generate_graph_from_adjacency_matrix_1(G, N, seed):
    """
    Arguments
    N: number of nodes
    """
    np.random.seed(seed)

    # Convert to np.ndArray
    A = nx.linalg.graphmatrix.adjacency_matrix(G).toarray() 
    nx.linalg
    #print("Adj: ", A, "\n", A.shape, "\n", type(A))

    # Different matrices
    D = np.sum(A, axis=0)
    D = np.diag(D)
    L = D - A
    invD = np.linalg.inv(D)
    invDA = A * invD
    invDL = invD * L
    invDLinvD = np.sqrt(invD) * L * np.sqrt(invD)
    
    
    Ln = nx.normalized_laplacian_matrix(G)
    Ln = Ln.toarray() # from sparse array to ndarray
    
    matrix = ['A', 'D', 'L', 'invD', 'invDA', 'invDL', 'invDinvD', 'Ln']
    matrices = [A, D, L, invD, invDA, invDL, invDLinvD, Ln]
    
    # Eigenvalues
    fig, axes = plt.subplots(3,3, figsize=(10,8))
    axes = axes.reshape(-1)
    fig.suptitle("Eigenvalues of various matrices")
    for i,m in enumerate(matrices): 
        ax = axes[i]
        eigs = np.linalg.eigvals(m)
        eigs = np.sort(eigs)[::-1] 
        ax.set_title(matrix[i])
        ax.grid(True)
        ax.plot(eigs, '-o')
    for i in range(i+2,axes.shape[-1]): axes[i].axis('off')
    plt.tight_layout()
    plt.show()

In [12]:
prob_slider = widgets.FloatSlider(min=0, max=1, step=0.1, value=0.5)
node_slider = widgets.IntSlider(min=3, max=30, step=1, value=10)
nb_neigh_slider = widgets.IntSlider(min=1, max=10, step=1, value=4)
nb_edges_per_node_slider = widgets.IntSlider(min=1, max=20, step=2, value=5)
seed_slider = widgets.IntSlider(int=1, max=50, step=1, value=25)
graph_type = ["connected_Hi Toan,

I just found out that you never switched your major to an M.S. project. Please do this ASAP. Otherwise you might have issues regarding graduation this semester or this summer. Karey can help you if needed. But you must ask. Thanks. This is your responsibility.

 Gordonwatts_strogatz", "powerlaw_cluster_graph"]


@interact_manual(
    nb_nodes=node_slider,
    prob=prob_slider,
    nb_neigh=nb_neigh_slider,
    nb_edges_per_node=nb_edges_per_node_slider,
    seed=seed_slider,
    graph_type=graph_type,
    # directed=True,
)
def drawGraph(nb_nodes, nb_neigh, prob, seed, nb_edges_per_node, graph_type):
    if graph_type == "connected_watts_strogatz":
        nb_edges_per_node_slider.style.handle_color = 'red'
        nb_neigh_slider.style.handle_color = 'black'
        nb_tries = 20
        edge_prob = prob
        G = nx.connected_watts_strogatz_graph(
            nb_nodes, nb_neigh, edge_prob, nb_tries, seed
        )
    elif graph_type == "powerlaw_cluster_graph":
        nb_neigh_slider.style.handle_color = 'red'
        nb_edges_per_node_slider.style.handle_color = 'black'
        add_tri_prob = prob
        if nb_edges_per_node >= nb_nodes:
            nb_edges_per_node = nb_nodes - 1
        G = nx.powerlaw_cluster_graph(nb_nodes, nb_edges_per_node, add_tri_prob, seed)

    generate_graph_from_adjacency_matrix_1(G, nb_nodes, seed)

interactive(children=(IntSlider(value=10, description='nb_nodes', max=30, min=3), IntSlider(value=4, descripti…

Hi Toan,

I just found out that you never switched your major to an M.S. project. Please do this ASAP. Otherwise you might have issues regarding graduation this semester or this summer. Karey can help you if needed. But you must ask. Thanks. This is your responsibility.

 GordonHi Toan,

I just found out that you never switched your major to an M.S. project. Please do this ASAP. Otherwise you might have issues regarding graduation this semester or this summer. Karey can help you if needed. But you must ask. Thanks. This is your responsibility.

 Gordon***
### Script added to polish the file

<script>
  $(document).ready(function(){
    $('div.prompt').hide();
    $('div.back-to-top').hide();
    $('nav#menubar').hide();
    $('.breadcrumb').hide();
    $('.hidden-print').hide();
  });
</script>

<footer id="attribution" style="float:right; color:#999; background:#fff;">
Created with Jupyter, delivered by Fastly, rendered by Rackspace.
</footer>

In [10]:
prob_slider = widgets.FloatSlider(min=0, max=1, step=0.1, value=0.5, description="Probability")
node_slider = widgets.IntSlider(min=3, max=20, step=1, value=7)
nb_neigh_slider = widgets.IntSlider(min=1, max=10, step=1, value=4)
nb_edges_per_node_slider = widgets.IntSlider(min=1, max=20, step=2, value=5)
seed_slider = widgets.IntSlider(int=1, max=50, step=1, value=25)
graph_type = ["connected_watts_strogatz", "powerlaw_cluster_graph", "circular_graph"]

# Also draw the eigenfunctions for the cyclic case where the nodes are arranged in a circular layout, 
# with labels in the nodes


@interact_manual(
    nb_nodes=node_slider,
    prob=prob_slider,
    nb_neigh=nb_neigh_slider,
    nb_edges_per_node=nb_edges_per_node_slider,
    seed=seed_slider,
    graph_type=graph_type,
)
def drawGraphEigenvalues(nb_nodes, nb_neigh, prob, seed, nb_edges_per_node, graph_type):
    if graph_type == "connected_watts_strogatz":
        nb_edges_per_node_slider.style.handle_color = "red"
        nb_neigh_slider.style.handle_color = "black"
        nb_tries = 20
        edge_prob = prob
        G = nx.connected_watts_strogatz_graph(
            nb_nodes, nb_neigh, edge_prob, nb_tries, seed
        )
    elif graph_type == "powerlaw_cluster_graph":
        nb_neigh_slider.style.handle_color = "red"
        nb_edges_per_node_slider.style.handle_color = "black"
        add_tri_prob = prob
        if nb_edges_per_node >= nb_nodes:
            nb_edges_per_node = nb_nodes - 1
        G = nx.powerlaw_cluster_graph(nb_nodes, nb_edges_per_node, add_tri_prob, seed)
    elif graph_type == "circular_graph":
        nb_neigh_slider.style.handle_color = "red"
        nb_edges_per_node_slider.style.handle_color = "red"
        nb_neigh_slider.style.handle_color = "red"
        prob_slider.style.handle_color = "red"
        seed_slider.style.handle_color = "red"

        G = nx.Graph()
        for n in range(nb_nodes):
            G.add_node(n)
        for n in range(nb_nodes):
            G.add_edge(n, n + 1)
        G.add_edge(nb_nodes - 1, 0)

    spec_lib.generate_eigenvectors_from_adjacency_matrix_1(G, nb_nodes, seed)

interactive(children=(IntSlider(value=7, description='nb_nodes', max=20, min=3), IntSlider(value=4, descriptio…

In [11]:
# Test Eigenfunction, sorting, etc. by creating a matrix whose eigenvalues I know
N_slider = widgets.IntSlider(min=3, max=10, step=1, value=5)
seed_slider = widgets.IntSlider(min=100, max=200, step=1)


@interact(N=N_slider, seed=seed_slider)
def test_eigen(N, seed):
    # generate eigenvalues
    np.random.seed(seed)
    # large variance for wider spread of spectrum
    eigens = (20.0 + 100.0 * np.random.randn(N)) / 20
    eigens = np.where(eigens < 0, -eigens, eigens)
    print("eigens= ", eigens)
    print("eigens[0]= ", eigens[0])
    print("eigens[1]= \n", eigens[1])
    # print("eigens= \n", eigens)
    eigens = np.diag(eigens)
    ee = np.linalg.eig(eigens)
    print("ee= \n",  ee)
    print("ee[0]= ", ee[0], type(ee[0]))
    print("ee[1]= \n", ee[1])
    
    args = np.argsort(ee[0])
    print("args:", args, type(args))
    ee0 = ee[0][args]
    ee1 = ee[1][:,args]
    print("sorted ee")
    print("ee[0]= ", ee0)
    print("ee[1]= \n", ee1)
    recursivelyrecursively
    
    # create eigenvectors
    x = ortho_group.rvs(N)
    # Similarity transform (eigenvalues of A are invariant)
    A = x.T @ eigens @ x
    # A = x @ np.linalg.inv(x)
    # print("A= \n", A)
    # print("x.T= \n", x.T)
    # print("inv(x)= \n", np.linalg.inv(x))
    eigens = np.linalg.eig(A)
    args = np.argsort(eigens[0])
    print("===============================")
    print("args: \n", args)
    eigs = eigens[0][args]
    print("unsorted eigs: \n", eigens[0])
    print("sorted eigs: \n", eigs)
    eigv = eigens[1][:,args]
    print("unsorted x:\n ", x.T)
    print("unsorted eigv: \n", eigens[1])
    print("sorted x: \n", x.T[:,args])
    print("sorted eigv= \n", eigv)

    pass

interactive(children=(IntSlider(value=5, description='N', max=10, min=3), IntSlider(value=100, description='se…

# Exploration of eigenvalue and eigenfunctions for the 1-D cyclic non-cyclic cases
As we have seen,  a signal $s^1=(s_0, s_1, \cdots, s_{N-1})\in\Re{N}$, is transformed into a signal $s^2\in\Re{N}$ by a filter $H$ according to 
$$ s^2 = H s^1$$  where $H$ is a matrix in $\Re{N\times N}$. Applying this filter recursively, one finds that 
\begin{align}
s^3 &= H s^2 \\
s^4 &= H s^3 \\
s^l &= H s^{l-1}
\end{align}
If this is done a large number of times, and if one assumes convergence of $s^l$ to a vector of finite norm, one finds in the limit: 
$$
s^\infty = H s^\infty
$$
which states that $s^\infty$ is an eigenvector of the filter $H$ with a unit eigenvalue $\lambda=1$. recursivelyrecursively